Übung: 3 VMs zu einem Kubernetes Cluster verbinden (k3s-Variante)
--------------------------------------------------

K3s bietet eine einfache und effiziente Möglichkeit, Kubernetes-Cluster lokal oder in Produktionsumgebungen zu betreiben. 

Das Hinzufügen von Nodes zu einem bestehenden K3s-Cluster erfolgt über den Befehl `curl -sfL https://get.k3s.io`. 

- - -

Um die Vorgänge zu visualisieren starten wird Kubernetes Dashboard und scrollen rechts herunter zu Cluster -> Nodes

Wählt dazu nachfolgenden Link an und aktzeptiert das Zertifikat um dann ohne Token, drückt "Überspringen" oder "Skip", ins Dashboard zu wechseln.


In [ ]:
! echo "https://"$(cat ~/work/server-ip)":30443"

---

###  Worker Nodes zur Control Plane hinzufügen

* Dazu holen wir uns zuerst den Token von der Control Plane Node
* Installieren k3s auf der Worker Node
* Joinen die Worker Node mit der Control Plane Node


In [ ]:
%%bash
export TOKEN=$(sudo cat /var/lib/rancher/k3s/server/node-token)
echo $TOKEN
ssh dukworker-11-default -- "curl -sfL https://get.k3s.io | sudo K3S_URL=https://$(cat ~/work/server-ip):6443 K3S_TOKEN=$TOKEN sh -"

In [ ]:
%%bash
export TOKEN=$(sudo cat /var/lib/rancher/k3s/server/node-token)
echo $TOKEN
ssh dukworker-12-default -- "curl -sfL https://get.k3s.io | sudo K3S_URL=https://$(cat ~/work/server-ip):6443 K3S_TOKEN=$TOKEN sh -"

In [ ]:
! kubectl get nodes 

- - -

### Zentraler Persistenter Speicher

Damit alle Kubernetes Nodes (Master + Worker) den gleichen zentralen Persistenten Speicher verwenden, wurder vorher auf dem Master ein NFS Share /data eingerichtet.

In [ ]:
! cat /etc/exports

Damit müssen sich jetzt noch die Worker Nodes verbinden

In [ ]:
! ssh dukworker-11-default -- "sudo mount -t nfs dukmaster-10-default:/data /data; df -h | grep /data"

In [ ]:
! ssh dukworker-12-default -- "sudo mount -t nfs dukmaster-10-default:/data /data; df -h | grep /data"

Die entsprechende Dateiablage, sogenannte PersistentVolumes (PV) in Kubernetes, wurde beim Installieren von Kubernetes eingerichtet. Das PersistentVolumes zeigt auf das Verzeichnis `/data`, siehe Eintrag `hostPath`.

Diese Einrichtung ermöglicht die Trennung von Speicher und Container-Lebenszyklus, sodass Daten über Container-(Pod)-Neustarts und -Löschungen hinaus bestehen bleiben.

In [ ]:
%%bash
kubectl get pv -o go-template='{{ range .items }}
apiVersion: v1
kind: PersistentVolume
metadata:
  name: {{ .metadata.name }}
spec:
  capacity:
    storage: {{ .spec.capacity.storage }}
  accessModes:
  {{- range .spec.accessModes }}
    - {{ . }}
  {{- end }}
  storageClassName: {{ .spec.storageClassName }}
  persistentVolumeReclaimPolicy: {{ .spec.persistentVolumeReclaimPolicy }}
  volumeMode: {{ .spec.volumeMode }}

  {{- if .spec.hostPath }}
  hostPath:
    path: {{ .spec.hostPath.path }}
    type: {{ .spec.hostPath.type }}
  {{- end }}

  {{- if .spec.claimRef }}
  claimRef:
    namespace: {{ .spec.claimRef.namespace }}
    name: {{ .spec.claimRef.name }}
  {{- end }}
status:
  phase: {{ .status.phase }}
---
{{ end }}'


Im Kubernetes Dashboard finden wir den Eintrag in dem wir rechts herunterscrollen auf Cluster -> PersistentVolumes

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":30443"